In [ ]:
import os

from weco_datascience.reporting import get_recent_data

In [ ]:
df = get_recent_data(config=os.environ, n=100000, index="metrics-conversion-prod")


In [ ]:
print(len(df))

What is the date range?

In [ ]:
df.head(1)

In [ ]:
df.tail(1)

Sort df by anonymousId and reindex

In [ ]:
sorted_df=df.sort_values(["anonymousId", "@timestamp"])
sorted_df2=sorted_df.reset_index(drop=True)

In [ ]:
sorted_df2.head(10)

1.Calculate the number of journeys from search to works pageviews

In [ ]:
anonId_prev=str("")
search_prev=str("")
search2works=0

for i in range(len(sorted_df2)):
    if ((sorted_df2.loc[i, "anonymousId"]!=anonId_prev) and (
        sorted_df2.loc[i, "page.name"]=="works") or (
        sorted_df2.loc[i, "page.name"]=="images") and (
        sorted_df2.loc[i, "page.query.query"]!="")):
            search_prev="yes" 
            anonId_prev=sorted_df2.loc[i, "anonymousId"]
    
    elif ((sorted_df2.loc[i, "anonymousId"]==anonId_prev) and (
        (sorted_df2.loc[i, "page.name"]=="work") or (
        (sorted_df2.loc[i, "page.name"]=="works"))) and (
        sorted_df2.loc[i,"page.query.id"]!="") and (
        search_prev=="yes")):
            search2works+=1
            search_prev=""
            anonId_prev=""
    else:
        search_prev=""
        anonId_prev=""
        
print("Total number of journeys from search to works pageviews: ", (search2works))
print("Total number of events: ", len(sorted_df2))
print("Percentage of journeys from search to works pageviews of total events: ", "{:.0%}".format(
    (search2works)/len(sorted_df2)))


2.Calculate the number of journeys from works pageviews to search 

In [ ]:
anonId_prev=str("")
works_prev=str("")
works2search=0


for i in range(len(sorted_df2)): 
    if ((sorted_df2.loc[i, "anonymousId"]!=anonId_prev) and (
        (sorted_df2.loc[i, "page.name"]=="work") or (
        sorted_df2.loc[i, "page.name"]=="works")) and (
        sorted_df2.loc[i,"page.query.id"]!="")):
            works_prev="yes"
            anonId_prev=sorted_df2.loc[i, "anonymousId"] 
    
    elif ((sorted_df2.loc[i, "anonymousId"]==anonId_prev) and (
        sorted_df2.loc[i, "page.name"]=="works") or (
        sorted_df2.loc[i, "page.name"]=="images") and (
        sorted_df2.loc[i, "page.query.query"]!="") and (
        works_prev=="yes")):
            works2search+=1
            works_prev=""
            anonId_prev=""

    else:
        search_prev=""
        anonId_prev=""
        
print("Total number of journeys from works pageviews to search: ", (works2search))
print("Total number of events: ", len(sorted_df2))
print("Percentage of works pageviews to search of total events: ", "{:.0%}".format(
    (works2search)/len(sorted_df2)))
print("\n")
print("Total number of journeys between works pageviews and search: ", (works2search + search2works))
print("Percentage of total journeys between works pagesviews and search out of total events: ", "{:.0%}".format(
    (works2search+search2works)/len(sorted_df2)))


3.Calculate the number of journeys from works pageviews to item page views 

In [ ]:
anonId_prev=str("")
works_prev=str("")
works2item=0


for i in range(len(sorted_df2)): 
    if ((sorted_df2.loc[i, "anonymousId"]!=anonId_prev) and (
        (sorted_df2.loc[i, "page.name"]=="work") or (
        sorted_df2.loc[i, "page.name"]=="works")) and (
        sorted_df2.loc[i,"page.query.id"]!="")):
            works_prev="yes"
            anonId_prev=sorted_df2.loc[i, "anonymousId"] 
    
    elif (((sorted_df2.loc[i, "anonymousId"]==anonId_prev) and (
        sorted_df2.loc[i, "page.name"]=="work") and (
        sorted_df2.loc[i, "page.pathname"]=="/item") and (
        sorted_df2.loc[i, "source"]!="viewer/paginator")) or (
        (sorted_df2.loc[i, "page.name"]=="images") and (
        sorted_df2.loc[i, "page.pathname"]=="/image") and (
        works_prev=="yes"))):
            works2item+=1
            works_prev=""
            anonId_prev=""
      
    else:
        works_prev=""
        anonId_prev=""
        
print("Total number of journeys from works pageviews to item pageviews: ", (works2item))
print("Total number of events: ", len(sorted_df2))
print("Percentage of journeys from works pageviews to item pageviews out of total events: ", "{:.0%}".format(
    (works2item)/len(sorted_df2)))


4.Calculate the number of journeys from item pageviews to works pageviews

In [ ]:
anonId_prev=str("")
item_prev=str("")
item2works=0


for i in range(len(sorted_df2)): 
    if ((sorted_df2.loc[i, "anonymousId"]!=anonId_prev) and ((
        (sorted_df2.loc[i, "page.name"]=="work") and (
        sorted_df2.loc[i, "page.pathname"]=="/item") and (
        sorted_df2.loc[i, "source"]!="viewer/paginator")) or (
        (sorted_df2.loc[i, "page.name"]=="images") and (
        sorted_df2.loc[i, "page.pathname"]=="/image")))):
            item_prev="yes"
            anonId_prev=sorted_df2.loc[i, "anonymousId"] 
    
    elif (((sorted_df2.loc[i, "anonymousId"]==anonId_prev) and (
        (sorted_df2.loc[i, "page.name"]=="work") or (
        sorted_df2.loc[i, "page.name"]=="works")) and (
        sorted_df2.loc[i,"page.query.id"]!="")) and (
        item_prev=="yes")):
            item2works+=1
            item_prev=""
            anonId_prev=""
    
    else:
        item_prev=""
        anonId_prev=""
        
print("Total number of journeys from item pageviews to works pageviews: ", (item2works))
print("Total number of events: ", len(sorted_df2))
print("Percentage of journeys from item pageviews to works pageviews out of total events: ", "{:.0%}".format(
    (item2works)/len(sorted_df2)))
print("\nTotal number of journeys between item and works pageviews: ", (
    works2item + item2works))
print("Percentage of total journeys between works and item pageviews out of total events: ", "{:.0%}".format(
    (works2item + item2works)/len(sorted_df2)))

5.Calculate the percentage of item pageviews by pagination out of the total number of events. Note that this code has been written to exclude those item pageviews from previous sessions which are still open in browser tabs causing them to fire on startup.

Note: images don't offer more than 1 page to view, so this analysis is for catalogue items only.

In [ ]:
anonId_prev=str(36)
itemPageview2pagination=0
itemPageview_prev=0


for i in range(len(sorted_df2)): 
    if ((sorted_df2.loc[i, "anonymousId"]!=anonId_prev) and ((
        (sorted_df2.loc[i, "page.name"]=="work") and (
        sorted_df2.loc[i, "page.pathname"]=="/item") and (
        sorted_df2.loc[i, "source"]!="viewer/paginator")) or (
        (sorted_df2.loc[i, "page.name"]=="images") and (
        sorted_df2.loc[i, "page.pathname"]=="/image")))):
            item_prev="yes"
            anonId_prev=sorted_df2.loc[i, "anonymousId"] 
            
    elif ((sorted_df2.loc[i, "anonymousId"]==anonId_prev) and (
        sorted_df2.loc[i, "page.name"]=="work") and (
        sorted_df2.loc[i, "page.pathname"]=="/item") and (
        sorted_df2.loc[i, "source"]=="viewer/paginator") and (
        item_prev=="yes")):
            itemPageview2pagination+=1
    
    else:
        item_prev=""
        anonId_prev=""
             

print("Total number of non-image item pageviews by pagination: ", itemPageview2pagination)
print("Total number of events: ", len(sorted_df2))
print("Percentage of non-image item pageviews by pagination out of total events: ", "{:.0%}".format(
    itemPageview2pagination/len(sorted_df2)))
